# Generate and Execute a Bash Script

This notebook builds on the [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb) recipe. Here, the generated Bash code is executed as a system command.

> **WARNING:** This recipe executes code generated by language models. The generated code may delete or modify files on your system. Use with caution!

See the [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb) recipe for instructions on installing the models you will need. It also has information on concepts like _system prompts_ and the differences between commands on different operating systems.



## Setting Up

### Install dependencies

Granite Kitchen comes with a bundle of dependencies that are required for Granite Cookbook recipes. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [1]:
!pip install git+https://github.com/ibm-granite-community/utils \
    langchain_community \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /private/var/folders/8t/m9m188_d0tb8szvfqlc20hfr0000gn/T/pip-req-build-4izgjftr
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /private/var/folders/8t/m9m188_d0tb8szvfqlc20hfr0000gn/T/pip-req-build-4izgjftr
  Resolved https://github.com/ibm-granite-community/utils to commit a5965f40db3950dd2a41f3ca62a2c34adcdc20d7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Select a model

Select a Granite Code model from the [`ibm-granite`](https://replicate.com/ibm-granite) org on Replicate. 

Here we use the Replicate Langchain client to connect to the model. To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).


In [2]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

model = Replicate(
    model="ibm-granite/granite-3.1-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
)

### Detect your operating system

In [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb), we used a Python helper function to determine the host operating system, MacOS, Linux, etc. We used this information in the prompts to encourage the model to generate shell commands that work on the host system, because some shell commands differ between operating systems. However, the output may still contain commands that are not supported by your operating system. Because this notebook attempts to run the generated commands, you may see failures if an incorrect command syntax was generated.

In [3]:
import os, platform

def os_name():
    os_name = platform.system()
    # It turns out, using "MacOS" is better than "Darwin", which is what gets returned on MacOS.
    # For all other cases, the returned value should be fine as is, so we map the result to the desired
    # name, but only for MacOS...
    name_map = {'Darwin': 'MacOS'}
    shell_map = {'Windows': 'DOS'} # On Windows and use Power Shell, change from `DOS` to `Power Shell`.
    # ... then pass the os_name value as the second arg, which is used as the default return value.
    # For the shell name, return `bash` by default. (You can change this to zsh, fish, etc.)
    return name_map.get(os_name, os_name), shell_map.get(os_name, 'bash')

In [4]:
my_os, my_shell = os_name()
my_os, my_shell

('MacOS', 'bash')

## Writing the Prompt for Direct Execution

### Write the System Prompt


First we want to modify the _system prompt_ we used in the previous notebook to work better for our purposes.

Writing prompts is an art. Recall in [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb), our output was usually Markdown with quoted sections of shell code and commentary explaining how it worked. Here, we just want code output that we execute without editing. Here are a few tips on writing prompts for our purposes here:

> **TIPS:**
>
> 1. Rather than use a question ("How do I ...?") in a prompt, provide a directive ("Write a script that ..."). This helps prevent the model from generating dialogue around the code.
> 2. Add instructions to the system prompt like this: "You are a helpful software engineer. You write clear, concise, well-commented code. You only print valid code. You don't print any commentary about the code nor markdown syntax to wrap the code."

So here is our new system prompt:

In [5]:
from textwrap import dedent
from langchain_core.messages import SystemMessage

system_prompt = SystemMessage(content=dedent(f"""\
    You are a helpful software engineer. You write clear, concise,
    well-commented code. You only print valid code. You don't print
    any commentary about the code nor do you wrap the code in markdown syntax!
    You make sure you only generate {my_shell} code that is {my_os}-compatible!
    Do not output anything but the code.
    """
))

### Provide a list of Q&A examples

One of the examples uses the `stat` command, which requires different syntax for Linux vs. MacOS systems.

In [6]:
stat_flags = '-c "%y %n" {}'
if my_os == 'MacOS':
    stat_flags = '-f "%m %N" {}'
print(f"The 'stat' flags for my OS \'{my_os}\' and \'{my_shell}\' are \'{stat_flags}\'")

The 'stat' flags for my OS 'MacOS' and 'bash' are '-f "%m %N" {}'


> **NOTE:** If you are using a Windows system, try changing the "answers" in the `examples` cell to be valid Power Shell or DOS commands. You can ignore the `stat_flags` in the next cell.

In [7]:
examples = [
    {
        "question": "Recursively find files that match '*.js', and filter out files with 'excludeddir' in their paths.",
        "answer": "find . -name '*.js' | grep -v excludeddir",
    },
    {
        "question": "Dump \"a0b\" as hexadecimal bytes.",
        "answer": "printf \"a0b\" | od -tx1",
    },
    {
        "question": "Create a tar ball of all pdf files in the current folder and any subdirectories.",
        "answer": "find . -name '*.pdf' | xargs tar czvf pdf.tar",
    },
    {
        "question": "Sort all files and directories in the current directory, but no subdirectories, according to modification time, and print only the seven most recently modified items.",
        "answer": f"find . -maxdepth 1 -exec stat {stat_flags} '\' | sort -n -r | tail -n 7",
    },
    {
        "question": "Find all the empty directories in and under the current directory.",
        "answer": "find . -type d -empty",
    },
]

<>:16: SyntaxWarning: invalid escape sequence '\;'
<>:16: SyntaxWarning: invalid escape sequence '\;'
/var/folders/8t/m9m188_d0tb8szvfqlc20hfr0000gn/T/ipykernel_24432/866531455.py:16: SyntaxWarning: invalid escape sequence '\;'
  "answer": f"find . -maxdepth 1 -exec stat {stat_flags} \; | sort -n -r | tail -n 7",


### Assemble the prompt template

Here we build up a chat prompt template from messages. See the [Langchain docs](https://python.langchain.com/docs/how_to/few_shot_examples_chat/#fixed-examples) for more details.

In [8]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

chat_template = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

print(chat_template.input_variables)


['question']


### View the completed prompt

Create a prompt and inspect the fully-interpolated chat template. Alternating Human/AI messages create a structure that the model will follow.

In [9]:
shell_prompt1 = dedent(f"""\
    Write a {my_shell} script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time
    for each file in the output.""").replace("\n", "")

print(chat_template.format(question=shell_prompt1))

System: You are a helpful software engineer. You write clear, concise,
well-commented code. You only print valid code. You don't print
any commentary about the code nor do you wrap the code in markdown syntax!
You make sure you only generate bash code that is MacOS-compatible!
Do not output anything but the code.

Human: Recursively find files that match '*.js', and filter out files with 'excludeddir' in their paths.
AI: find . -name '*.js' | grep -v excludeddir
Human: Dump "a0b" as hexadecimal bytes.
AI: printf "a0b" | od -tx1
Human: Create a tar ball of all pdf files in the current folder and any subdirectories.
AI: find . -name '*.pdf' | xargs tar czvf pdf.tar
Human: Sort all files and directories in the current directory, but no subdirectories, according to modification time, and print only the seven most recently modified items.
AI: find . -maxdepth 1 -exec stat -f "%m %N" {} \; | sort -n -r | tail -n 7
Human: Find all the empty directories in and under the current directory.
AI: 

### Run the model - example 1

In [10]:
chain = chat_template | model
shell_code1 = chain.invoke({"question": shell_prompt1})
print(shell_code1)

#!/bin/bash

# Find files modified within the last week
find . -type f -newerwt -print0 | head -z -n 50

# Print last modification time for each file
while IFS= read -r -d '' file; do
    echo -n "$(date -r "$file" +"%Y-%m-%d %H:%M:%S")"
    echo "$file"
done < <(find . -type f -newerwt -print0) | head -z -n 50


Compare this output to what you got in the [../Text_to_Shell](../Text_to_Shell/Text_to_Shell.ipynb) recipe. Is this output a valid script and nothing else? Or, is there extra commentary and Markdown formatting? If you got this extra, undesirable output, try running the cell again. Does modifying the prompt or system prompt help?

Now we can attempt to execute the script! There is no need for an additional helper function:

In [11]:
os.system(shell_code1)

find: -newerwt: unknown primary or operator
head: invalid option -- z
usage: head [-n lines | -c bytes] [file ...]
sh: -c: line 9: syntax error near unexpected token `<'
sh: -c: line 9: `done < <(find . -type f -newerwt -print0) | head -z -n 50'


512

### Run the model - example 2

Let's try another one.

In [12]:
shell_code2 = chain.invoke({
    "question": f"""Write a {my_shell} script to recursively find Jupyter notebooks
in the parent directory and print their paths."""
})

print(shell_code2)

#!/bin/bash

find ../ -type f -name "*.ipynb" -exec echo {} \;


In [13]:
os.system(shell_code2)

..//Auto_Documentation/Auto_Documentation.ipynb
..//Fine_Tuning_Granite/Finetuning_Granite.ipynb
..//Unit_Tests_Generation/Unit_Tests_Generation.ipynb
..//Guard-Rails/HAP.ipynb
..//Text_to_SQL/Text_to_SQL.ipynb
..//Text_to_SQL/TwitterDataset/TwitterSyntheticDataGen.ipynb
..//Text_to_Shell/Text_to_Shell.ipynb
..//Code_Gen_from_Tests/Code_Gen_from_Tests.ipynb
..//CodeLLM_DevKit/validating_code_translation.ipynb
..//CodeLLM_DevKit/code_summarization.ipynb
..//CodeLLM_DevKit/generate_unit_tests.ipynb
..//Coding_Assistant_in_VSCode/Coding_Assistant_in_VSCode.ipynb
..//Text_to_Shell_Exec/Text_to_Shell_Exec.ipynb
..//Getting_Started_with_Granite_Code.ipynb


0

Try invoking the model several times. How do the responses change from one invocation to the next? Try different queries. adding more examples to the `examples` string or modifying the ones shown. Does this affect the outputs?